In [40]:
from langchain.tools import Tool
import pandas as pd
import matplotlib.pyplot as plt
import uuid
import re
import os

os.makedirs("plots", exist_ok=True)

def single_input_visualization_tool(input_str: str) -> str:
    try:
        # Parse the input
        file_match = re.search(r"file\s*:\s*(\S+)", input_str)
        query_match = re.search(r"query\s*:\s*(.+)", input_str)

        if not file_match or not query_match:
            return "❌ Please provide input as 'file: <path> query: <your query>'."

        file_path = file_match.group(1)
        query = query_match.group(1)

        if file_path.endswith(".csv"):
            df = pd.read_csv(file_path)
        elif file_path.endswith(".json"):
            df = pd.read_json(file_path)
        else:
            return "❌ Unsupported file format."

        # Detect columns from query
        def extract_column_names(query, columns):
            return [col for col in columns if re.search(rf"\b{re.escape(col)}\b", query, re.IGNORECASE)]

        cols = extract_column_names(query, df.columns)

        # Detect plot type
        if "histogram" in query.lower():
            if len(cols) < 1:
                return "❌ Provide at least one column for histogram."
            col = cols[0]
            plt.figure()
            df[col].dropna().hist(bins=20)
            plt.title(f"Histogram of {col}")
            plt.xlabel(col)
            plt.ylabel("Frequency")

        elif "scatter" in query.lower():
            if len(cols) < 2:
                return "❌ Provide two columns for scatterplot."
            x, y = cols[:2]
            plt.figure()
            plt.scatter(df[x], df[y], alpha=0.7)
            plt.title(f"{x} vs {y}")
            plt.xlabel(x)
            plt.ylabel(y)
        else:
            return "❌ Specify 'histogram' or 'scatterplot' in your query."

        filename = f"plots/plot_{uuid.uuid4().hex[:8]}.png"
        plt.savefig(filename)
        plt.close()
        return f"✅ Plot saved at: {filename}"

    except Exception as e:
        return f"❌ Error: {str(e)}"

# Convert to LangChain Tool
visualization_tool = Tool(
    name="visualization_tool",
    func=single_input_visualization_tool,
    description="Generate histogram or scatterplot from a dataset. Input format: 'file: <file_path> query: <your query>'",
)


In [42]:
from langchain.agents import initialize_agent, AgentType


In [44]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("deepseek-r1-distill-llama-70b", model_provider="groq")

In [46]:
tools = [visualization_tool]

In [48]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)


In [50]:
query = "file: assignment_data.csv query: Create a scatterplot of orders_all_np vs DisplayAll_spend"
response = agent.run(query)
print(response)




> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `<think>
Alright, I'm trying to figure out how to create a scatterplot of 'orders_all_np' vs 'DisplayAll_spend' using the 'assignment_data.csv' file. I remember that the visualization tool can help with this. I should format the input correctly for the visualization_tool function. It needs to be in the format 'file: <file_path> query: <your_query>'. 

So, the file path is 'assignment_data.csv', and the query is to make a scatterplot comparing those two specific columns. I should make sure to specify 'scatterplot' and the exact column names. I think the input string should look like 'file: assignment_data.csv query: scatterplot of orders_all_np vs DisplayAll_spend'. 

I hope the tool can handle that and generate the plot without any issues. Let me go ahead and input that.
